In [1]:
from torchvision.models.resnet import ResNet18_Weights, ResNet34_Weights, ResNet50_Weights, ResNet101_Weights, ResNet152_Weights
import torch
import torch.nn as nn

In [2]:
%load_ext autoreload
%autoreload 1
%aimport resnet_with_stochastic_depth
from resnet_with_stochastic_depth import resnet18, resnet34, resnet50, resnet101, resnet152

We are able to load resnet's checkpoint pretrained on ImageNet1k to our resnet with stochastic depth feature. When the dataset's num_classes differs from ImageNet1k, we don't load fc layer's state_dict but initialize a new linear layer.

In [3]:
model = resnet18(ResNet18_Weights.DEFAULT, 101)

We can see the effect of stochastic depth, forward feature flow randomly entirely bypasses transformations of residual connected blocks determined by a hyperparameter p. And the network's output varies even if the input is the same.

In [4]:
model.train()
inputs = torch.randn(10, 3, 224, 224)
model(inputs)

tensor([[-0.2071,  0.2988,  0.1513,  ..., -0.8192,  0.5240,  0.0231],
        [ 0.0261, -0.0500,  0.0075,  ..., -0.0058, -0.0127, -0.0376],
        [-0.0205, -0.0114,  0.0083,  ..., -0.0071, -0.0325, -0.0791],
        ...,
        [-0.4531,  1.0337, -0.4057,  ...,  1.0673, -0.6579,  0.2893],
        [ 1.0489, -0.7404,  0.0095,  ...,  0.7564,  0.0701,  0.7026],
        [ 0.1505, -0.3629,  0.1278,  ...,  0.5193, -0.1565, -0.0288]],
       grad_fn=<AddmmBackward0>)

In [5]:
model(inputs)

tensor([[-0.0247,  0.0027, -0.0539,  ..., -0.0631, -0.0714, -0.1090],
        [ 0.0140, -0.0215, -0.1087,  ...,  0.0362,  0.0015, -0.0206],
        [-0.0053, -0.0561, -0.0457,  ...,  0.0164,  0.0038, -0.0056],
        ...,
        [-0.3896,  1.4654, -0.1463,  ...,  0.0784, -0.3896,  0.6980],
        [ 0.2780,  0.8620,  0.6639,  ...,  0.7295, -1.4380,  1.4261],
        [-0.4696, -0.0999,  0.5681,  ...,  0.4733,  0.0966, -0.2582]],
       grad_fn=<AddmmBackward0>)

In [6]:
model(inputs)

tensor([[ 0.4142, -0.4380, -0.2854,  ...,  0.1313, -0.5269,  0.5528],
        [ 0.0704, -0.3012,  0.3292,  ...,  0.1617, -0.3061,  0.7049],
        [ 0.3585, -0.4288,  0.6654,  ...,  0.0135,  0.1588,  0.6313],
        ...,
        [ 0.0026, -0.0797,  0.5283,  ...,  0.4522,  0.1258,  0.4067],
        [-0.0657,  0.7774, -0.3234,  ...,  0.7168, -0.6277,  0.4707],
        [-0.0361,  0.0397, -0.0150,  ...,  0.0016, -0.0088, -0.0393]],
       grad_fn=<AddmmBackward0>)

But if we switch the model to evaluation mode, transformations in residual blocks will not be bypassed and the model's behavior is stable.

In [7]:
model.eval()
model(inputs)

tensor([[-0.2872, -0.0030,  0.7586,  ..., -0.0365,  0.0843, -0.5997],
        [-0.1706,  0.0584,  0.6666,  ..., -0.0843,  0.4512, -0.6301],
        [-0.1639, -0.0172,  0.6245,  ..., -0.1691,  0.1937, -0.6822],
        ...,
        [-0.1049, -0.0702,  0.7032,  ..., -0.1880,  0.2427, -0.6062],
        [-0.2370, -0.0245,  0.7233,  ..., -0.3226,  0.2937, -0.7685],
        [-0.0843, -0.0572,  0.7462,  ..., -0.2880,  0.2679, -0.7726]],
       grad_fn=<AddmmBackward0>)

In [8]:
model(inputs)

tensor([[-0.2872, -0.0030,  0.7586,  ..., -0.0365,  0.0843, -0.5997],
        [-0.1706,  0.0584,  0.6666,  ..., -0.0843,  0.4512, -0.6301],
        [-0.1639, -0.0172,  0.6245,  ..., -0.1691,  0.1937, -0.6822],
        ...,
        [-0.1049, -0.0702,  0.7032,  ..., -0.1880,  0.2427, -0.6062],
        [-0.2370, -0.0245,  0.7233,  ..., -0.3226,  0.2937, -0.7685],
        [-0.0843, -0.0572,  0.7462,  ..., -0.2880,  0.2679, -0.7726]],
       grad_fn=<AddmmBackward0>)

We can check that the stochastic depth module's parameter p is not in the state_dict, if you don't use register_parameter to explicity add it into state_dict. And we intended to keep p out of the state_dict, such that other people can load checkpoints trained in stochastic depth way into their resnet model

In [9]:
model.state_dict().keys()

odict_keys(['conv1.weight', 'bn1.weight', 'bn1.bias', 'bn1.running_mean', 'bn1.running_var', 'bn1.num_batches_tracked', 'layer1.0.conv1.weight', 'layer1.0.bn1.weight', 'layer1.0.bn1.bias', 'layer1.0.bn1.running_mean', 'layer1.0.bn1.running_var', 'layer1.0.bn1.num_batches_tracked', 'layer1.0.conv2.weight', 'layer1.0.bn2.weight', 'layer1.0.bn2.bias', 'layer1.0.bn2.running_mean', 'layer1.0.bn2.running_var', 'layer1.0.bn2.num_batches_tracked', 'layer1.1.conv1.weight', 'layer1.1.bn1.weight', 'layer1.1.bn1.bias', 'layer1.1.bn1.running_mean', 'layer1.1.bn1.running_var', 'layer1.1.bn1.num_batches_tracked', 'layer1.1.conv2.weight', 'layer1.1.bn2.weight', 'layer1.1.bn2.bias', 'layer1.1.bn2.running_mean', 'layer1.1.bn2.running_var', 'layer1.1.bn2.num_batches_tracked', 'layer2.0.conv1.weight', 'layer2.0.bn1.weight', 'layer2.0.bn1.bias', 'layer2.0.bn1.running_mean', 'layer2.0.bn1.running_var', 'layer2.0.bn1.num_batches_tracked', 'layer2.0.conv2.weight', 'layer2.0.bn2.weight', 'layer2.0.bn2.bias', '

We can also using resnet checkpoints to warmly initialize our other resnet types with stochastic depth features

In [10]:
res34 = resnet34(ResNet34_Weights.IMAGENET1K_V1, 101)
res34(inputs)

tensor([[-2.1859,  1.0355,  0.0076,  ..., -0.2601, -0.4679, -0.0733],
        [-3.2096,  0.1139,  0.6609,  ...,  0.5881, -0.4603,  0.6689],
        [-1.0280,  0.1074,  0.6715,  ..., -0.2442, -0.2563,  0.2256],
        ...,
        [-1.6409,  0.4982,  0.9146,  ..., -1.1320, -1.4986, -0.4490],
        [-0.7836,  0.4552,  0.6676,  ...,  0.5659, -1.2078,  0.4879],
        [-1.7799, -0.3279, -0.0044,  ..., -0.0726, -0.5789, -0.4745]],
       grad_fn=<AddmmBackward0>)

In [11]:
res50 = resnet50(ResNet50_Weights.IMAGENET1K_V1, 101)
res50(inputs)

tensor([[-0.0239,  0.1816,  0.3410,  ...,  0.0391,  0.1266, -0.3540],
        [-0.5165,  0.0149,  0.3084,  ...,  0.0732,  0.4344, -0.4949],
        [-0.6280, -0.0973,  0.8390,  ...,  0.2119, -0.0086, -0.8956],
        ...,
        [-0.7209,  0.0081,  0.7862,  ..., -0.3381,  0.2873, -0.5396],
        [-0.0879, -0.0516,  0.1623,  ..., -0.0074,  0.1340, -1.2557],
        [-0.5529, -0.0988,  0.5948,  ..., -0.3012,  0.3844, -0.7722]],
       grad_fn=<AddmmBackward0>)

In [12]:
res101 = resnet101(ResNet101_Weights.IMAGENET1K_V1, 101)
res101(inputs)

tensor([[-0.1583, -0.1834,  0.2267,  ...,  0.1559,  0.0200,  0.2109],
        [-0.2338,  0.0781,  0.1152,  ...,  0.0320,  0.0765, -0.0304],
        [ 0.1106, -0.0155,  0.1023,  ...,  0.0871, -0.3275, -0.0366],
        ...,
        [-0.1106, -0.1279,  0.2189,  ...,  0.2921,  0.0841, -0.0409],
        [-0.3478, -0.2100,  0.0992,  ...,  0.3578, -0.0451,  0.0192],
        [-0.0381, -0.2339,  0.1748,  ...,  0.4624,  0.4405,  0.1017]],
       grad_fn=<AddmmBackward0>)

In [13]:
res152 = resnet152(ResNet152_Weights.IMAGENET1K_V1, 101)
res152(inputs)

tensor([[-0.0220,  0.1416, -0.0066,  ..., -0.2603, -0.0251,  0.0138],
        [ 0.0755,  0.1352,  0.2472,  ..., -0.1418, -0.0724, -0.1352],
        [ 0.2736,  0.1197,  0.5599,  ..., -0.1255,  0.1535,  0.2896],
        ...,
        [ 0.1921,  0.0391,  0.1435,  ..., -0.1785,  0.4594, -0.0784],
        [ 0.2485,  0.1313,  0.5552,  ..., -0.1589,  0.1998,  0.0142],
        [ 0.1407, -0.1176,  0.5173,  ..., -0.2292,  0.2347,  0.0364]],
       grad_fn=<AddmmBackward0>)